In [2]:
import pandas as pd
import numpy as np
import math
from math import sqrt
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.neighbors import NearestNeighbors

# Knn incremental sample sizes
(Calculating metrics for different sample sizes, for different values of k, except for pt.3, as it takes too long and we could only train it once.)
1. finding optimal value of k and calculating metrics on ~5000 ratings by fitting model on train data (train)
2. finding optimal value of k and calculating metrics on ~5000 ratings by fitting model on whole data except for the data for which rating is to be predicted (train + test - removed ratings)
3. predicting ~5,00,000 ratings

# 1. Trying to find optimal k for ~ 5000 ratings

In [425]:
sampled_ratings=pd.read_csv("data/final_sample.csv") 
sampled_ratings.drop("bin",axis=1, inplace=True)
sr=sampled_ratings

In [426]:
sr = sr.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [427]:
sr_train, sr_test = tts(sr, test_size=.2, random_state = 69)

In [428]:
sr_train.shape

(800, 137232)

In [429]:
sr_test.shape

(200, 137232)

In [432]:
sr_test_list = sr_test.unstack().reset_index(name="rating").set_index("movieId")
sr_test_list = sr_test_list[sr_test_list.rating>0]


In [433]:
sr_test_list.head()
sr_test_list.shape

(2496213, 2)

In [383]:
x, sr_test_rem = tts(sr_test_list, test_size=0.01, random_state = 69)

In [413]:
len(sr_test_rem)

4471

In [385]:
a = list(sr_test_rem.index)
b = list(sr_test_rem.loc[:, "userId"])

for i in range(len(a)):
    sr_test.at[a[i],b[i]]=0

In [393]:
def fit_model():
    model_knn = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=k, n_jobs=-1)
    model_knn.fit(sr_train)
    
def recommendation(mid, k):
    if int(sr_test[sr_test.index == mid].sum(axis=1)) == 0:
        results = pd.DataFrame(list(zip([0]*k,[0]*k)), columns = ['sim', 'mov_id'])
    else:
        query_index=mid
        distances,indices=model_knn.kneighbors(sr_test[sr_test.index==query_index].values.reshape(1,-1))
        ind = list(indices.flatten())
        sim=[]
        for i in range(len(distances)):
            sim.append(1-float(distances[i]))
        mov_ids = []
        for i in ind:
            mov_ids.append(pr.index[i])
        results = pd.DataFrame(list(zip(sim,mov_ids)), columns = ['sim', 'mov_id'])
        results = results.set_index("mov_id")
    return results

In [395]:
def cal_mean(s):
    count = 0
    add = 0 
    for i in range(len(s)):
        if s[i] != 0:
            add += s[i]
            count += 1
    return float(add)/(count+0.001)

def predict(mid, uid, r):
    w_sum = 0 
    sim_list = []
    if max(list(r.loc[:, "sim"]))==0:
        return cal_mean(list(sr_train[uid]))
    else:
        for m in list(r.index):
            if float(sr_train[sr_train.index == m][uid]) != 0.0:
                sim_list.append(float(r[r.index == m]['sim']))
                w_sum += float(sr_train[sr_train.index == m][uid])*float(r[r.index == m]['sim'])
        if sim_list == []:
            return cal_mean(list(sr_train[uid]))
        else:
            return w_sum/sum(sim_list)    

4.0

In [421]:
def multi_k(k):
    act = []
    pred = []
    i=0
    m_list = list(set(sr_test_rem.index))

    for m in m_list:
        u_list = list(sr_test_rem[sr_test_rem.index==m]['userId'])
        r = recommendation(m, k)
        for u in u_list:
            p = predict(m, u, r)
            act.append(float(sr_test_rem[(sr_test_rem.index == m)&(sr_test_rem['userId'] ==u)]['rating']))
            pred.append(p)
            
    rmse = sqrt(mean_squared_error(act, pred))
    mae = mean_absolute_error(act, pred)
    print(k, " - RMSE:", rmse, " - MAE:", mae)

In [423]:
for i in range(1,20):
    multi_k(i)

1  - RMSE: 1.1321851831325704  - MAE: 0.8369749353168224
2  - RMSE: 1.124617327636748  - MAE: 0.8251358298340683
3  - RMSE: 1.1214272083385344  - MAE: 0.8214906157621026
4  - RMSE: 1.121796338003273  - MAE: 0.8232049165339123
5  - RMSE: 1.1099648518622782  - MAE: 0.8149228984580854
6  - RMSE: 1.100608662898046  - MAE: 0.8096278927787601
7  - RMSE: 1.0997300658855695  - MAE: 0.8099050131430698
8  - RMSE: 1.0829942395022307  - MAE: 0.7986570115126314
9  - RMSE: 1.0789688283085876  - MAE: 0.7955897913569666
10  - RMSE: 1.0690718444363385  - MAE: 0.7874965639473441
11  - RMSE: 1.0695042080040698  - MAE: 0.7902998166842402
12  - RMSE: 1.0684580884942187  - MAE: 0.7887762344842753
13  - RMSE: 1.0693443736154586  - MAE: 0.7905791827179033
14  - RMSE: 1.0720046976975666  - MAE: 0.7943892476085018
15  - RMSE: 1.0720649390800703  - MAE: 0.7926909454698724
16  - RMSE: 1.0725841281295863  - MAE: 0.7931371693737066
17  - RMSE: 1.0713514528836436  - MAE: 0.7930644029328154
18  - RMSE: 1.071778143520

# 2. Trying to find optimal k for ~ 5000 ratings by fitting whole data

In [514]:
sampled_ratings=pd.read_csv("data/final_sample.csv") 
sampled_ratings.drop("bin",axis=1, inplace=True)
sr=sampled_ratings

In [515]:
sr = sr.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [516]:
sr_train, sr_test = tts(sr, test_size=.2, random_state = 69)

In [517]:
sr_train.shape

(800, 137232)

In [518]:
sr_test.shape

(200, 137232)

In [519]:
sr_test_list = sr_test.unstack().reset_index(name="rating").set_index("movieId")
sr_test_list = sr_test_list[sr_test_list.rating>0]

In [520]:
sr_test_list.head()
sr_test_list.shape

(447076, 2)

In [521]:
x, sr_test_rem = tts(sr_test_list, test_size=0.01, random_state = 69)

In [522]:
len(sr_test_rem)

4471

In [523]:
a = list(sr_test_rem.index)
b = list(sr_test_rem.loc[:, "userId"])

for i in range(len(a)):
    sr.at[a[i],b[i]]=0

In [531]:
def goodness_of_fit(predictions,ratings):
  ss_res = 0
  for i in range(len(predictions)):
    err = (ratings[i]-predictions[i])**2
    ss_res += err
  ss_tot = 0
  for i in range(len(ratings)):
    ss_tot += (ratings[i]-np.mean(ratings))**2
  r2 = 1 - (ss_res / ss_tot)
  return r2

In [532]:
def fit_model(k):
    model_knn = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=k, n_jobs=-1)
    model_knn.fit(sr)
    return model_knn

def recommendation(model, mid, k):
    if int(sr[sr.index == mid].sum(axis=1)) == 0:
        results = pd.DataFrame(list(zip([0]*k,[0]*k)), columns = ['sim', 'mov_id'])
    else:
        query_index=mid
        distances,indices=model.kneighbors(sr[sr.index==query_index].values.reshape(1,-1))
        ind = list(indices.flatten())
        mov_ids = []
        sim=[]
        for i in range(len(distances)):
            sim.append(1-float(distances[i]))
        mov_ids = []
        for i in ind:
            mov_ids.append(pr.index[i])
        results = pd.DataFrame(list(zip(sim,mov_ids)), columns = ['sim', 'mov_id'])
        results = results.set_index("mov_id")

    return results

In [533]:
def cal_mean(s):
    count = 0
    add = 0 
    for i in range(len(s)):
        if s[i] != 0:
            add += s[i]
            count += 1
    return float(add)/(count+0.001)

def predict(mid, uid, r):
    w_sum = 0 
    sim_list = []
    if max(list(r.loc[:, "sim"]))==0:
        return cal_mean(list(sr[uid]))
    else:
        for m in list(r.index):
            if float(sr[sr.index == m][uid]) != 0.0:
                sim_list.append(float(r[r.index == m]['sim']))
                w_sum += float(sr[sr.index == m][uid])*float(r[r.index == m]['sim'])
        if sim_list == []:
            return cal_mean(list(sr[uid]))
        else:
            return w_sum/sum(sim_list)


In [536]:
def multi_k(k):
    model = fit_model(k)
    act = []
    pred = []
    i=0
    m_list = list(set(sr_test_rem.index))

    for m in m_list:
        u_list = list(sr_test_rem[sr_test_rem.index==m]['userId'])
        r = recommendation(model, m, k)
        for u in u_list:
            p = predict(m, u, r)
            act.append(float(sr_test_rem[(sr_test_rem.index == m)&(sr_test_rem['userId'] ==u)]['rating']))
            pred.append(p)
            
    r2 = goodness_of_fit(pred, act)
    rmse = sqrt(mean_squared_error(act, pred))
    mae = mean_absolute_error(act, pred)
    print(k, " - RMSE:", rmse, " - MAE:", mae, " - R squared:", r2)
    return k, rmse, mae, r2

In [537]:
rmse_list = []
mae_list = []
k_list = []
r2_list = []
k_vals = [1,2,4,8,16,32,64,128]

for i in k_vals:
    a, b, c, d = multi_k(i)
    rmse_list.append(b)
    mae_list.append(c)
    k_list.append(a)
    r2_list.append(d)

1  - RMSE: 1.0601122710672  - MAE: 0.8168132790761045  - R squared: 0.05848462536752019
2  - RMSE: 1.1141850105606568  - MAE: 0.8242908356085275  - R squared: -0.0400119178087428
4  - RMSE: 1.1098681286754766  - MAE: 0.8135760652165854  - R squared: -0.03196852981141385
8  - RMSE: 1.0866472105585039  - MAE: 0.8044571225335627  - R squared: 0.01076190583744796
16  - RMSE: 1.0429698204767806  - MAE: 0.776407709266831  - R squared: 0.08868781606319098
32  - RMSE: 1.0340175774008826  - MAE: 0.7752444580421411  - R squared: 0.10426501693989088
64  - RMSE: 1.0352529479648556  - MAE: 0.7802506514230283  - R squared: 0.10212341765240651
128  - RMSE: 1.0432545247944869  - MAE: 0.7944249887976628  - R squared: 0.08819021791470494


In [538]:
min_rmse = 99999999999
min_k = 0

for i in range(len(rmse_list)):
    if rmse_list[i] < min_rmse:
        min_rmse = rmse_list[i]
        min_k = k_list[i]

# 3. Predicting ~ 5,00,000 ratings

In [55]:
import numpy as np

In [539]:
sampled_ratings=pd.read_csv("data/final_sample.csv") #reading csv sample rating
sampled_ratings.drop("bin",axis=1, inplace=True)
sr=sampled_ratings

In [540]:
sr = sr.pivot(
    index='movieId',
    columns='userId',
    values='rating'
).fillna(0)

In [541]:
sr_test_list = sr.unstack().reset_index(name="rating").set_index("movieId")
sr_test_list = sr_test_list[sr_test_list.rating>0]

In [542]:
sr_test_list.shape

(2496213, 2)

In [543]:
x, sr_test_rem = tts(sr_test_list, test_size=0.2, random_state = 69)

In [544]:
len((sr_test_rem.index))

499243

In [545]:
a = list(sr_test_rem.index)
b = list(sr_test_rem.loc[:, "userId"])

for i in range(len(a)):
    sr.at[a[i],b[i]]=0

In [553]:
def fit_model(k):
    model_knn = NearestNeighbors(metric='cosine',algorithm='brute', n_neighbors=k, n_jobs=-1)
    model_knn.fit(sr)
    return model_knn
    
def recommendation(model, mid, k):
    if int(sr[sr.index == mid].sum(axis=1)) == 0:
        results = pd.DataFrame(list(zip([0]*k,[0]*k)), columns = ['sim', 'mov_id'])
    else:
        query_index=mid
        distances,indices=model.kneighbors(sr[sr.index==query_index].values.reshape(1,-1))
        ind = list(indices.flatten())
        distances = list(distances.flatten())
        sim=[]
        for i in range(len(distances)):
            sim.append(1-float(distances[i]))
        mov_ids = []
        for i in ind:
            mov_ids.append(sr.index[i])
        results = pd.DataFrame(list(zip(sim,mov_ids)), columns = ['sim', 'mov_id'])
        results = results.set_index("mov_id")
    return results

In [555]:
def cal_mean(s):
    count = 0
    add = 0 
    for i in range(len(s)):
        if s[i] != 0:
            add += s[i]
            count += 1
    return float(add)/(count+0.001)

def predict(mid, uid, r):
    w_sum = 0 
    sim_list = []
    if max(list(r.loc[:, "sim"]))==0:
        return cal_mean(list(sr[uid]))
    else:
        for m in list(r.index):
            if float(sr[sr.index == m][uid]) != 0.0:
                sim_list.append(float(r[r.index == m]['sim']))
                w_sum += float(sr[sr.index == m][uid])*float(r[r.index == m]['sim'])
        if sim_list == []:
            return cal_mean(list(sr[uid]))
        else:
            return w_sum/sum(sim_list)    

In [556]:
def multi_k(k):
    model = fit_model(k)
    act = []
    pred = []
    i=0
    m_list = list(set(sr_test_rem.index))

    for m in m_list:
        u_list = list(sr_test_rem[sr_test_rem.index==m]['userId'])
        print(i, "of", "499243 completed")
        i=i+len(u_list)
        r = recommendation(model, m, k)
        for u in u_list:
            p = predict(m, u, r)
            act.append(float(sr_test_rem[(sr_test_rem.index == m)&(sr_test_rem['userId'] ==u)]['rating']))
            pred.append(p)
            
    r2 = goodness_of_fit(pred, act)
    rmse = sqrt(mean_squared_error(act, pred))
    mae = mean_absolute_error(act, pred)
    print(k, " - RMSE:", rmse, " - MAE:", mae, " - R squared:", r2)
    return pred, act

### Considering the trade-off between run time and improvement in RMSE, taking min_k = 8

In [562]:
min_k = 8
p, a = multi_k(min_k)

data = pd.DataFrame(list(zip(p, a)), columns=['pred', 'act'])
data.to_csv("recomm_resultk]s.csv", sep=',')

0 of 499243 completed
1 of 499243 completed
2164 of 499243 completed
3019 of 499243 completed
3030 of 499243 completed
3077 of 499243 completed
3080 of 499243 completed
3086 of 499243 completed
3117 of 499243 completed
6867 of 499243 completed
7200 of 499243 completed
11392 of 499243 completed
11480 of 499243 completed
11544 of 499243 completed
11552 of 499243 completed
13134 of 499243 completed
13155 of 499243 completed
14964 of 499243 completed
15776 of 499243 completed
16284 of 499243 completed
16340 of 499243 completed
16364 of 499243 completed
17946 of 499243 completed
19255 of 499243 completed
19833 of 499243 completed
19836 of 499243 completed
19928 of 499243 completed
19942 of 499243 completed
21258 of 499243 completed
21259 of 499243 completed
23137 of 499243 completed
23167 of 499243 completed
26646 of 499243 completed
26812 of 499243 completed
26855 of 499243 completed
26870 of 499243 completed
27047 of 499243 completed
29323 of 499243 completed
29652 of 499243 completed
311

220087 of 499243 completed
220088 of 499243 completed
220547 of 499243 completed
220589 of 499243 completed
220612 of 499243 completed
220627 of 499243 completed
220637 of 499243 completed
220675 of 499243 completed
220678 of 499243 completed
220684 of 499243 completed
220685 of 499243 completed
220687 of 499243 completed
222003 of 499243 completed
223418 of 499243 completed
224959 of 499243 completed
224960 of 499243 completed
225003 of 499243 completed
225008 of 499243 completed
225010 of 499243 completed
225032 of 499243 completed
225033 of 499243 completed
225628 of 499243 completed
225683 of 499243 completed
225710 of 499243 completed
225735 of 499243 completed
229904 of 499243 completed
229906 of 499243 completed
229907 of 499243 completed
230302 of 499243 completed
230433 of 499243 completed
231159 of 499243 completed
231218 of 499243 completed
232945 of 499243 completed
232951 of 499243 completed
232954 of 499243 completed
234282 of 499243 completed
234283 of 499243 completed
2

402969 of 499243 completed
402972 of 499243 completed
403117 of 499243 completed
403381 of 499243 completed
403383 of 499243 completed
403412 of 499243 completed
403581 of 499243 completed
403589 of 499243 completed
405828 of 499243 completed
405923 of 499243 completed
405926 of 499243 completed
406613 of 499243 completed
406638 of 499243 completed
406652 of 499243 completed
406664 of 499243 completed
407261 of 499243 completed
407328 of 499243 completed
408003 of 499243 completed
408004 of 499243 completed
408080 of 499243 completed
408201 of 499243 completed
408462 of 499243 completed
408463 of 499243 completed
408465 of 499243 completed
408960 of 499243 completed
408961 of 499243 completed
409160 of 499243 completed
409336 of 499243 completed
409540 of 499243 completed
409552 of 499243 completed
409611 of 499243 completed
409731 of 499243 completed
409740 of 499243 completed
411165 of 499243 completed
411452 of 499243 completed
414516 of 499243 completed
414578 of 499243 completed
4